In [16]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model

from mol_opt.mol_opt import MolOpt
from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, get_latest_model
from mol_opt.ot_utils import compute_barycenter

from rdkit.Chem import MolFromSmiles

import torch
import numpy as np
import time
import torch.nn.functional as F

/home/octav/gitrepos/tum-thesis
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
model_type = "pointwise"
sys.argv = ["", "-cuda", "-model_type", model_type, "-one_batch_train"]
args = get_args()
outdir_suffix = "dev1"
args.n_epochs = 1000 
args.init_model = "{}-test5".format(model_type)
args.init_decoder_model = "{}_decode".format(args.init_model)
args.output_dir = "mol_opt/output_{}/{}".format(outdir_suffix, args.init_model)
args.tb_logs_dir = "mol_opt/logs_{}/{}".format(outdir_suffix, args.init_model)
args.batch_size = 50 

args.annealing_rate = 0.002
args.connectivity_lambda = 0
args.valency_lambda = 0
args.euler_lambda = 1
args.connectivity = False 
args.valency = True 
args.euler_characteristic_penalty = True
args.scale_lambdas = False
args.conn_penalty_function = "logdet" 
args

Namespace(N_transformer=6, agg_func='sum', annealing_rate=0.002, batch_norm=False, batch_size=50, conn_penalty_function='logdet', connectivity=False, connectivity_hard=False, connectivity_lambda=0, cuda=True, device='cuda:0', dim_tangent_space=40, dropout_ffn=0.0, dropout_gcn=0.0, dropout_transformer=0.1, euler_characteristic_penalty=True, euler_lambda=1, ffn_activation='LeakyReLU', init_decoder_model='pointwise-test5_decode', init_model='pointwise-test5', linear_out=False, model_type='pointwise', n_epochs=1000, n_ffn_hidden=100, n_ffn_transformer=100, n_heads_transformer=10, n_hidden=250, n_labels=1, n_layers=5, one_batch_train=True, ot_solver='emd', output_dir='mol_opt/output_dev1/pointwise-test5', pc_hidden=100, penalty_gumbel=False, pred_hidden=150, scale_lambdas=False, sinkhorn_entropy=0.1, sinkhorn_max_it=10000, task='qed', tb_logs_dir='mol_opt/logs_dev1/pointwise-test5', valency=True, valency_hard=False, valency_lambda=0)

In [21]:
(1-args.annealing_rate) ** 1000

0.13506452244668338

In [22]:
import os
os.path.join("~/Downloads", "")

'~/Downloads/'

In [23]:
train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, same_number_atoms = True)
# val_data_loader = get_loader("iclr19-graph2graph/data/qed", "valid", 36, False)

In [ ]:
molopt, molopt_decoder = main(args, train_data_loader = train_data_loader, val_data_loader = None)

No model pointwise-test5 found in mol_opt/output_dev1/pointwise-test5! Starting from scratch.
Epoch: 1
Penalty params: tau=1.00000 conn_l=0.00000 val_l=0.00000 euler_l=1.00000 epoch=1
Losses Epoch 1, train
 fgw:3.7897064
 conn_penalty:0.0000000
 val_penalty:567.3843750
 euler_penalty:156.8166992
 total:160.6064062
Measure Epoch 1, train
 similarity:0.0000000
 penlog:-100.0000000
Metrics Epoch 1, train
 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:11.4921223
 batch_molecular_disconnected_validity:0.0000000
 batch_connected_components:1.0000000
 batch_invalid_valency_nodes:66.6357739
 batch_nodes_0degree:0.0000000
 batch_nodes_7plus_degree:21.5800000
 invalid_euler_toofew:0.0000000
 invalid_euler_toomany:100.0000000
 avg_euler_error:214.1300000
 batch_node_degree:21.8637627
{'train': {'avg_euler_error': ['Margin', ['train_metrics/avg_euler_error/mean', 'train_metrics/avg_euler_error/bot_band', 'train_metrics/avg_euler_error/top_band']], 'batch_no

Penalty params: tau=0.98018 conn_l=0.00000 val_l=0.00000 euler_l=1.00000 epoch=11
Losses Epoch 11, train
 fgw:1.8868159
 conn_penalty:0.0000000
 val_penalty:21.4971924
 euler_penalty:20.5429565
 total:22.4297729
Measure Epoch 11, train
 similarity:0.0000000
 penlog:4.8874377
Metrics Epoch 11, train
 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:74.6987952
 batch_molecular_disconnected_validity:100.0000000
 batch_connected_components:21.5800000
 batch_invalid_valency_nodes:100.0000000
 batch_nodes_0degree:21.5800000
 batch_nodes_7plus_degree:0.0000000
 invalid_euler_toofew:100.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-21.5800000
 batch_node_degree:0.0000000
Epoch duration: 2.6034247875213623
mol_opt/output_dev1/pointwise-test5/model_pointwise-test5_11
Epoch: 12
Penalty params: tau=0.97822 conn_l=0.00000 val_l=0.00000 euler_l=1.00000 epoch=12
Losses Epoch 12, train
 fgw:2.4996072
 conn_penalty:0.0000000
 val_penalty:21.5796997
 eu

Penalty params: tau=0.96075 conn_l=0.00000 val_l=0.00000 euler_l=1.00000 epoch=21
Losses Epoch 21, train
 fgw:4.2939380
 conn_penalty:0.0000000
 val_penalty:21.5800000
 euler_penalty:20.5800000
 total:24.8739380
Measure Epoch 21, train
 similarity:0.0000000
 penlog:4.8874377
Metrics Epoch 21, train
 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:74.6987952
 batch_molecular_disconnected_validity:100.0000000
 batch_connected_components:21.5800000
 batch_invalid_valency_nodes:100.0000000
 batch_nodes_0degree:21.5800000
 batch_nodes_7plus_degree:0.0000000
 invalid_euler_toofew:100.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-21.5800000
 batch_node_degree:0.0000000
Epoch duration: 2.545912027359009
mol_opt/output_dev1/pointwise-test5/model_pointwise-test5_21
Epoch: 22
Penalty params: tau=0.95883 conn_l=0.00000 val_l=0.00000 euler_l=1.00000 epoch=22
Losses Epoch 22, train
 fgw:4.1441931
 conn_penalty:0.0000000
 val_penalty:21.5800000
 eul

Penalty params: tau=0.94171 conn_l=0.00000 val_l=0.00000 euler_l=1.00000 epoch=31
Losses Epoch 31, train
 fgw:1.8939639
 conn_penalty:0.0000000
 val_penalty:17.6116846
 euler_penalty:2.3352206
 total:4.2291846
Measure Epoch 31, train
 similarity:0.0029412
 penlog:-46.8177169
Metrics Epoch 31, train
 batch_molecular_validity:0.0000000
 batch_correctness:0.0000000
 batch_symbol_accuracy:81.9277108
 batch_molecular_disconnected_validity:52.0000000
 batch_connected_components:17.7000000
 batch_invalid_valency_nodes:100.0000000
 batch_nodes_0degree:16.8600000
 batch_nodes_7plus_degree:0.2400000
 invalid_euler_toofew:98.0000000
 invalid_euler_toomany:0.0000000
 avg_euler_error:-15.4200000
 batch_node_degree:0.5708990
Epoch duration: 2.5525472164154053
mol_opt/output_dev1/pointwise-test5/model_pointwise-test5_31
Epoch: 32
Penalty params: tau=0.93982 conn_l=0.00000 val_l=0.00000 euler_l=1.00000 epoch=32
Losses Epoch 32, train
 fgw:1.6989757
 conn_penalty:0.0000000
 val_penalty:21.4728076
 eule

In [29]:
for i in train_data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break
    
x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize(*yhat_logits)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

from mol_opt.ot_utils import FGW 
fgw_loss = FGW(alpha = 0.5)
fgw_loss(*pred_pack, tau = 1)

tensor(27.7234, device='cuda:0', grad_fn=<AddBackward0>)

In [37]:
bonds_logits[0:22**2, :].view(22,22,-1).shape

torch.Size([22, 22, 5])

In [40]:
bonds_labels[0:22**2].view(22,22)[5]

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
       device='cuda:0')

In [32]:
from tensorboardX import SummaryWriter
writer = SummaryWriter("/home/octav/gitrepos/tum-thesis/mol_opt/dev7")

In [31]:
tags = [
    "pointwise10-dev8/train_avg_euler_error/mean",
    "pointwise10-dev8/train_avg_euler_error/bot_band",
    "pointwise10-dev8/train_avg_euler_error/top_band"
]
# writer.add_custom_scalars_marginchart(tags)


In [2]:
# writer.add_scalar("plm2/plm2", 0, 1)
# writer.add_scalar("plm2/plm2", 1, 2)
# writer.add_scalar("plm2/plm2", 0, 3)
# writer.add_scalar("plm2/plm2", 1, 4)
tags = ["plm", "plm", "plm"]
# layout = {"plm" : {"plm" : ["Margin", tags]}}
layout = {"plm1" : {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]},
          "plm2": {"plm" : ["Margin", tags], "plm2": ["Multiline", tags]}}
writer.add_custom_scalars(layout)

In [33]:
writer._get_file_writer().add_summary()

TypeError: add_summary() missing 1 required positional argument: 'summary'

In [34]:
writer.logdir

'/home/octav/gitrepos/tum-thesis/mol_opt/dev6'

In [11]:
writer.close()

In [4]:
import tensorboardX
tensorboardX.__version__

'2.1'

In [6]:
from tensorboardX.summary import custom_scalars

In [16]:
writer._get_file_writer().add_summary(custom_scalars(layout), global_step = 3)
writer.flush()

In [34]:
writer.add_scalars('plm', {'plm0' : 0.0, 'plmt' : 0.1, 'plmb' : -0.1}, 3)
writer.flush()

In [ ]:
tau = 1
symbols_nll, charges_nll, bonds_nll = F.gumbel_softmax(tau = tau, dim=1, logits = symbols_logits), F.gumbel_softmax(tau=tau,dim=1,logits=charges_logits), F.gumbel_softmax(tau=tau,dim=1, logits = bonds_logits)

In [ ]:
np.sort(symbols_nll.mean(axis = 0).detach().numpy())

In [ ]:
list(molopt.opt0.parameters())[0]

In [ ]:
molopt.state_dict()

In [ ]:
model_dict = torch.load('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8')

In [ ]:
model_dict["model"]['opt0.weight']

In [ ]:
molopt2 = MolOpt(args)

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt2.load_state_dict(model_dict["model"])

In [ ]:
list(molopt2.opt0.parameters())[0]

In [ ]:
molopt3,args3 = load_model('mol_opt/output_pointwise10-onebatch/model_pointwise10-onebatch_8', MolOpt, args.device)

In [ ]:
list(molopt3.opt0.parameters())[0]

In [ ]:
fgw_loss(*pred_pack, tau = 1)